In [1]:
!pip install sklearn_crfsuite

   ---------------------------------------- 0.0/155.8 kB ? eta -:--:--
   -- ------------------------------------- 10.2/155.8 kB ? eta -:--:--
   ------- -------------------------------- 30.7/155.8 kB 1.4 MB/s eta 0:00:01
   ---------- ---------------------------- 41.0/155.8 kB 495.5 kB/s eta 0:00:01
   -------------------- ------------------ 81.9/155.8 kB 573.4 kB/s eta 0:00:01
   -------------------- ------------------ 81.9/155.8 kB 573.4 kB/s eta 0:00:01
   --------------------------- ---------- 112.6/155.8 kB 437.6 kB/s eta 0:00:01
   ----------------------------- -------- 122.9/155.8 kB 450.6 kB/s eta 0:00:01
   -------------------------------------  153.6/155.8 kB 459.5 kB/s eta 0:00:01
   -------------------------------------- 155.8/155.8 kB 424.4 kB/s eta 0:00:00


In [2]:
import nltk, re, pprint
import numpy as np
import pandas as pd
import pprint, time
import random
from sklearn.model_selection import train_test_split
from nltk.tokenize import word_tokenize
import sklearn

import sklearn_crfsuite
from sklearn import metrics
from sklearn_crfsuite import scorers
from sklearn.model_selection import GridSearchCV

import matplotlib.pyplot as plt

In [5]:
nltk.download('brown')
nltk.download('universal_tagset')
nltk_data = list(nltk.corpus.brown.tagged_sents(tagset='universal'))

[nltk_data] Downloading package brown to
[nltk_data]     C:\Users\KIIT0001\AppData\Roaming\nltk_data...
[nltk_data]   Package brown is already up-to-date!
[nltk_data] Downloading package universal_tagset to
[nltk_data]     C:\Users\KIIT0001\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping taggers\universal_tagset.zip.


In [6]:
print(nltk_data[1])
print(len(nltk_data))

[('The', 'DET'), ('jury', 'NOUN'), ('further', 'ADV'), ('said', 'VERB'), ('in', 'ADP'), ('term-end', 'NOUN'), ('presentments', 'NOUN'), ('that', 'ADP'), ('the', 'DET'), ('City', 'NOUN'), ('Executive', 'ADJ'), ('Committee', 'NOUN'), (',', '.'), ('which', 'DET'), ('had', 'VERB'), ('over-all', 'ADJ'), ('charge', 'NOUN'), ('of', 'ADP'), ('the', 'DET'), ('election', 'NOUN'), (',', '.'), ('``', '.'), ('deserves', 'VERB'), ('the', 'DET'), ('praise', 'NOUN'), ('and', 'CONJ'), ('thanks', 'NOUN'), ('of', 'ADP'), ('the', 'DET'), ('City', 'NOUN'), ('of', 'ADP'), ('Atlanta', 'NOUN'), ("''", '.'), ('for', 'ADP'), ('the', 'DET'), ('manner', 'NOUN'), ('in', 'ADP'), ('which', 'DET'), ('the', 'DET'), ('election', 'NOUN'), ('was', 'VERB'), ('conducted', 'VERB'), ('.', '.')]
57340


In [8]:
train_set,test_set = train_test_split(nltk_data,train_size=0.85,test_size=0.15,random_state=101)
train_set,val_set = train_test_split(nltk_data,train_size=0.80,test_size=0.20,random_state=101)

In [9]:
def word_features(sent, i):
    word = sent[i][0]
    pos = sent[i][1]
    
    
    if i==0:
        prevword = '<START>'
        prevpos = '<START>'
    else:
        prevword = sent[i-1][0]
        prevpos = sent[i-1][1]
        
    
    if i==0 or i==1:
        prev2word = '<START>'
        prev2pos = '<START>'
    else:
        prev2word = sent[i-2][0]
        prev2pos = sent[i-2][1]
    
    
    if i == len(sent)-1:
        nextword = '<END>'
        nextpos = '<END>'
    else:
        nextword = sent[i+1][0]
        nextpos = sent[i+1][1]
    
    
    pref_1, pref_2, pref_3, pref_4 = word[:1], word[:2], word[:3], word[:4]
    suff_1, suff_2, suff_3, suff_4 = word[-1:], word[-2:], word[-3:], word[-4:]
    

    
    return {'word':word,            
            'prevword': prevword,
            'prevpos': prevpos,  
            'nextword': nextword, 
            'nextpos': nextpos,          
            'suff_1': suff_1,  
            'suff_2': suff_2,  
            'suff_3': suff_3,  
            'suff_4': suff_4, 
            'pref_1': pref_1,  
            'pref_2': pref_2,  
            'pref_3': pref_3, 
            'pref_4': pref_4,
            'prev2word': prev2word,
            'prev2pos': prev2pos           
           }  

In [10]:
print(train_set[0][0:5])

word_features(train_set[0],3)

[('A', 'DET'), ('Newfoundland', 'NOUN'), ('sat', 'VERB'), ('solemnly', 'ADV'), ('beside', 'ADP')]


{'word': 'solemnly',
 'prevword': 'sat',
 'prevpos': 'VERB',
 'nextword': 'beside',
 'nextpos': 'ADP',
 'suff_1': 'y',
 'suff_2': 'ly',
 'suff_3': 'nly',
 'suff_4': 'mnly',
 'pref_1': 's',
 'pref_2': 'so',
 'pref_3': 'sol',
 'pref_4': 'sole',
 'prev2word': 'Newfoundland',
 'prev2pos': 'NOUN'}

In [16]:
def sent2features(sent):
    return[word_features(sent,i)for i in range(len(sent))]
def sent2labels(sent):
    return[postag for word,postag in sent]
def sent2tokens(sent):
    return[word for word,postag in sent]


In [17]:
 
X_train = [sent2features(s) for s in train_set]
y_train = [sent2labels(s) for s in train_set]

X_valid = [sent2features(s) for s in val_set]
y_valid = [sent2labels(s) for s in val_set]

X_test = [sent2features(s) for s in test_set]
y_test = [sent2labels(s) for s in test_set]


In [18]:
print(X_train[0][0:10])
print(y_train[0][0:10])

[{'word': 'A', 'prevword': '<START>', 'prevpos': '<START>', 'nextword': 'Newfoundland', 'nextpos': 'NOUN', 'suff_1': 'A', 'suff_2': 'A', 'suff_3': 'A', 'suff_4': 'A', 'pref_1': 'A', 'pref_2': 'A', 'pref_3': 'A', 'pref_4': 'A', 'prev2word': '<START>', 'prev2pos': '<START>'}, {'word': 'Newfoundland', 'prevword': 'A', 'prevpos': 'DET', 'nextword': 'sat', 'nextpos': 'VERB', 'suff_1': 'd', 'suff_2': 'nd', 'suff_3': 'and', 'suff_4': 'land', 'pref_1': 'N', 'pref_2': 'Ne', 'pref_3': 'New', 'pref_4': 'Newf', 'prev2word': '<START>', 'prev2pos': '<START>'}, {'word': 'sat', 'prevword': 'Newfoundland', 'prevpos': 'NOUN', 'nextword': 'solemnly', 'nextpos': 'ADV', 'suff_1': 't', 'suff_2': 'at', 'suff_3': 'sat', 'suff_4': 'sat', 'pref_1': 's', 'pref_2': 'sa', 'pref_3': 'sat', 'pref_4': 'sat', 'prev2word': 'A', 'prev2pos': 'DET'}, {'word': 'solemnly', 'prevword': 'sat', 'prevpos': 'VERB', 'nextword': 'beside', 'nextpos': 'ADP', 'suff_1': 'y', 'suff_2': 'ly', 'suff_3': 'nly', 'suff_4': 'mnly', 'pref_1':

In [19]:
crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs',
    c1=0.01,
    c2=0.1,
    max_iterations=100,
    all_possible_transitions=True   
)
try:
    crf.fit(X_train, y_train)
except AttributeError:
    pass

In [20]:
labels = list(crf.classes_)

In [22]:
y_pred = crf.predict(X_valid)


from sklearn_crfsuite import metrics as mt

mt.flat_f1_score(y_valid, y_pred,average='weighted', labels=labels)
                      

0.9815135950857892

In [23]:
from sklearn_crfsuite.utils import flatten
y_true_flat=flatten(y_valid)
y_pred_flat=flatten(y_pred)
sorted_labels=sorted(
    labels,
    key=lambda name:(name[1:],name[0])
)
print(metrics.classification_report(
    y_true_flat, y_pred_flat, labels=sorted_labels, digits=3
))

              precision    recall  f1-score   support

           .      1.000     1.000     1.000     29701
           X      0.953     0.813     0.878       300
         ADJ      0.950     0.940     0.945     16852
         ADP      0.981     0.985     0.983     29177
         ADV      0.952     0.941     0.946     11383
        VERB      0.983     0.983     0.983     37018
         DET      0.994     0.996     0.995     27692
        CONJ      0.995     0.997     0.996      7655
        NOUN      0.979     0.984     0.982     55378
        PRON      0.992     0.989     0.990     10032
         PRT      0.954     0.951     0.952      6074
         NUM      0.988     0.992     0.990      2838

    accuracy                          0.982    234100
   macro avg      0.977     0.964     0.970    234100
weighted avg      0.982     0.982     0.982    234100

